In [153]:
from tqdm.auto import tqdm
import pandas as pd
from macrohet import tile, dataio, tools
import numpy as np
import pickle
import zarr
import btrack
import os
import json

### Extract metadata

In [9]:
expt_ID = 'PS0000'
base_dir = f'/mnt/DATA/macrohet/{expt_ID}/'
metadata_fn = os.path.join(base_dir, 'acquisition/Images/Index.idx.xml')
metadata = dataio.read_harmony_metadata(metadata_fn)

Reading metadata XML file...


Extracting HarmonyV5 metadata:   0%|          | 0/113400 [00:00<?, ?it/s]

Extracting metadata complete!


In [10]:
metadata_path = os.path.join(base_dir, 'acquisition/Assaylayout/20210602_Live_cell_IPSDMGFP_ATB.xml')
assay_layout = dataio.read_harmony_metadata(metadata_path, assay_layout=True,)# mask_exist=True,  image_dir = image_dir, image_metadata = metadata)

Reading metadata XML file...
Extracting metadata complete!


### Choose position and extract tracks and true IDs

In [25]:
acq_ID = row, column = (3, 5)
tracks_fn = f'/mnt/DATA/macrohet/PS0000/labels/macrohet_seg_model/{acq_ID}.h5'
dict_fn = f'/mnt/DATA/macrohet/{expt_ID}/upstream_development/tracking/tracking_performance/v1>70/{row},{column}/{acq_ID}_track_assessment.json'

In [21]:
with open(dict_fn) as json_data:
    track_dict = json.load(json_data)
true_IDs = [int(ID) for ID in track_dict.keys() if track_dict[ID] == True]

In [22]:
with btrack.io.HDF5FileHandler(tracks_fn, 'r', obj_type='obj_type_1') as reader:
    tracks = reader.tracks
    masks = reader.segmentation
true_tracks = [t for t in tracks if t.ID in true_IDs]

[INFO][2023/12/19 02:10:56 PM] Opening HDF file: /mnt/DATA/macrohet/PS0000/labels/macrohet_seg_model/(3, 5).h5...
INFO:btrack.io.hdf:Opening HDF file: /mnt/DATA/macrohet/PS0000/labels/macrohet_seg_model/(3, 5).h5...
[INFO][2023/12/19 02:10:56 PM] Loading tracks/obj_type_1
INFO:btrack.io.hdf:Loading tracks/obj_type_1
[INFO][2023/12/19 02:10:56 PM] Loading LBEP/obj_type_1
INFO:btrack.io.hdf:Loading LBEP/obj_type_1
[INFO][2023/12/19 02:10:56 PM] Loading objects/obj_type_1 (41424, 5) (41424 filtered: None)
INFO:btrack.io.hdf:Loading objects/obj_type_1 (41424, 5) (41424 filtered: None)
[INFO][2023/12/19 02:11:07 PM] Loading segmentation (75, 6048, 6048)
INFO:btrack.io.hdf:Loading segmentation (75, 6048, 6048)
[INFO][2023/12/19 02:11:07 PM] Closing HDF file: /mnt/DATA/macrohet/PS0000/labels/macrohet_seg_model/(3, 5).h5
INFO:btrack.io.hdf:Closing HDF file: /mnt/DATA/macrohet/PS0000/labels/macrohet_seg_model/(3, 5).h5


### Load images and sc_df

In [66]:
image_dir = os.path.join(base_dir, 'acquisition/Images')
images = tile.compile_mosaic(image_dir, 
                             metadata, 
                             row, column, 
                             set_plane='max_proj',
                             set_channel=2,
                            )
images = images[:,0,...].compute().compute()

array([[[  0,   0,   0, ...,   0,   0,   0],
        [112, 109, 103, ..., 104, 103,   0],
        [105, 108, 109, ..., 107, 104,   0],
        ...,
        [115, 116, 112, ..., 111, 108,   0],
        [117, 118, 109, ..., 115, 111,   0],
        [118, 112, 101, ..., 108, 111,   0]],

       [[  0,   0,   0, ...,   0,   0,   0],
        [114, 105, 103, ..., 107, 108,   0],
        [108, 104, 106, ..., 111, 111,   0],
        ...,
        [114, 110, 110, ..., 111, 114,   0],
        [108, 112, 107, ..., 113, 106,   0],
        [108, 107, 102, ..., 108, 111,   0]],

       [[  0,   0,   0, ...,   0,   0,   0],
        [109, 105, 103, ..., 109, 107,   0],
        [110, 106, 103, ..., 110, 103,   0],
        ...,
        [115, 118, 113, ..., 114, 115,   0],
        [115, 118, 117, ..., 113, 111,   0],
        [118, 122, 114, ..., 105, 108,   0]],

       ...,

       [[  0,   0,   0, ...,   0,   0,   0],
        [116, 105, 105, ..., 111, 109,   0],
        [107, 106, 105, ..., 111, 108,   0

In [284]:
df = pd.read_pickle(f'/mnt/DATA/macrohet/{expt_ID}/results/preliminary_sc_measures/sc_dfs/{acq_ID}_sc_df_GT_dev.pkl')

In [285]:
df

,Time (hours),dMtb,Strain,Compound,Concentration,Unique ID,Acquisition ID,Cell ID,Mphi Area,Mphi GFP,...,%dMtb/dt,%dMtb,Mean Mtb Intensity,Thresholded Mean Mtb Intensity,Infection Status,Initial Infection Status,Final Infection Status,"dMtb/dt,T=15","Max. dMtb/dt,T=15",Mtb Area µm2
0,0,136.772588,WT,CTRL,EC0,1.3.5,"(3, 5)",1,29567.0,2220.025391,...,0.000000,292.26361,473.168976,167.325424,True,True,True,NaN,6.314185,46.797680
1,1,136.772588,WT,CTRL,EC0,1.3.5,"(3, 5)",1,26181.0,2549.692383,...,4.106972,292.26361,498.701813,202.661972,True,True,True,NaN,6.314185,48.719647
2,2,136.772588,WT,CTRL,EC0,1.3.5,"(3, 5)",1,26043.0,2653.458740,...,7.155963,292.26361,502.468353,210.382874,True,True,True,NaN,6.314185,52.206007
3,3,136.772588,WT,CTRL,EC0,1.3.5,"(3, 5)",1,25605.0,2736.911621,...,-3.167808,292.26361,501.905182,212.070969,True,True,True,NaN,6.314185,50.552221
4,4,136.772588,WT,CTRL,EC0,1.3.5,"(3, 5)",1,26430.0,2490.947266,...,7.736516,292.26361,502.564423,204.565842,True,True,True,NaN,6.314185,54.463202
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
296255,70,0.000000,WT,CTRL,EC0,99.3.5,"(3, 5)",99,47470.0,4803.345215,...,0.000000,0.00000,388.573425,0.000000,True,True,True,0.000000,0.591489,0.000000
296256,71,0.000000,WT,CTRL,EC0,99.3.5,"(3, 5)",99,41936.0,5695.538086,...,inf,0.00000,409.822296,21.552986,True,True,True,0.591489,0.591489,8.872339
296257,72,0.000000,WT,CTRL,EC0,99.3.5,"(3, 5)",99,53247.0,4450.872559,...,-100.000000,0.00000,387.400879,0.000000,True,True,True,0.000000,0.591489,0.000000
296258,73,0.000000,WT,CTRL,EC0,99.3.5,"(3, 5)",99,43669.0,5377.374512,...,0.000000,0.00000,393.975342,0.000000,True,True,True,0.000000,0.591489,0.000000


In [30]:
len(df['Cell ID'].unique()), len(true_IDs)

(170, 384)

In [177]:
already_done = set(list(df['Cell ID'].unique()))
true_IDs = set(true_IDs)
to_do_IDs = true_IDs.difference(already_done)

In [190]:
track_IDs = to_do_IDs
mtb_areas_dict = {}
dfs = []
for track_ID in tqdm(track_IDs, total = len(track_IDs)):
    track = [t for t in tracks if t.ID == track_ID][0]
    info = assay_layout.loc[acq_ID]
    mtb_areas = tools.measure_mtb_area(track, masks, rfp_images)
    # initial infection status 
    # init_infec_status = [True for t in track.t if any(mtb_areas[0:5])]
    ### compile single track dictionary of info
    d = {'Time (hours)':track['t'], 
         'Mtb Area µm2': mtb_areas,
         'x':track['x'],
         'y':track['y'],
         # 'Infection status':track['Infected'],
         # 'Initial infection status':track['Infected'][0], 
         # 'Final infection status':track['Infected'][-1], 
         'Mphi Area':track['area'], 
         'Mtb RFP':track['mean_intensity'][:, 1],
         'Mtb RFP threshold':track['mean_intensity'][:, 2],
         'Mphi GFP':track['mean_intensity'][:, 0],
         'Eccentricity':np.sqrt(1-((track['minor_axis_length']**2)/(track['major_axis_length']**2))),
         'MSD': [tools.euc_dist(track['x'][i-1], 
                                track['y'][i-1], 
                                track['x'][i], 
                                track['y'][i]) 
                  if i != 0 else 0
                  for i in range(0, len(track))],
         'Strain':[info['Strain'] for i in range(len(track['t']))], 
         'Compound':[info['Compound'] for i in range(len(track['t']))], 
         'Concentration':[info['ConcentrationEC'] for i in range(len(track['t']))], 
         'Cell ID':[track.ID for i in range(len(track['t']))],
         'Acquisition ID':[acq_ID for i in range(len(track['t']))], 
         'Unique ID': [f'{track.ID}.{acq_ID[0]}.{acq_ID[1]}.' for i in range(len(track['t']))]}

    mtb_areas_dict[f'{track.ID}.{acq_ID}'] = mtb_areas
    ### append df to list of dfs
    dfs.append(pd.DataFrame(d))
### concat single track dfs into big df
df2 = pd.concat(dfs, ignore_index=True)
#rename rd1
df2['Strain/Compound'] = df2['Compound']  # Create a new column and copy values from 'Compound'
# Replace values in 'NewColumn' where 'Strain' is 'RD1' with 'RD1'
df2.loc[df2['Strain'] == 'RD1', 'Strain/Compound'] = 'RD1'
# save out 
df2.to_pickle(f'/mnt/DATA/macrohet/{expt_ID}/results/preliminary_sc_measures/sc_dfs/{acq_ID}_sc_df_GT_dev_otherGT_trax.pkl')

  0%|          | 0/214 [00:00<?, ?it/s]

Calculating mtb area for every frame in track: 1536:   0%|          | 0/12 [00:00<?, ?it/s]

Calculating mtb area for every frame in track: 2563:   0%|          | 0/39 [00:00<?, ?it/s]

Calculating mtb area for every frame in track: 1539:   0%|          | 0/30 [00:00<?, ?it/s]

Calculating mtb area for every frame in track: 524:   0%|          | 0/69 [00:00<?, ?it/s]

...

Calculating mtb area for every frame in track: 1003:   0%|          | 0/37 [00:00<?, ?it/s]

Calculating mtb area for every frame in track: 493:   0%|          | 0/69 [00:00<?, ?it/s]

Calculating mtb area for every frame in track: 1007:   0%|          | 0/25 [00:00<?, ?it/s]

Calculating mtb area for every frame in track: 499:   0%|          | 0/37 [00:00<?, ?it/s]

Calculating mtb area for every frame in track: 1015:   0%|          | 0/70 [00:00<?, ?it/s]

### Merge dfs

In [282]:
df1 = df
# Optional: Ensure the column order is the same in both dataframes
df2 = df2[df1.columns]

# Concatenate dataframes
df = pd.concat([df1, df2], ignore_index=True)

In [283]:
df

,Time (hours),Strain,Compound,Concentration,Unique ID,Acquisition ID,Cell ID,Mphi Area,Mphi GFP,x,y,Eccentricity,MSD,Technical Replicate,Strain/Compound,Mtb Area µm2,Mtb RFP,Mtb RFP thresholded
0,0,WT,CTRL,EC0,1.3.5,"(3, 5)",1,29567.0,2220.025391,519.922607,876.779602,0.859159,0.000000,1,CTRL,46.797680,473.168976,167.325424
1,1,WT,CTRL,EC0,1.3.5,"(3, 5)",1,26181.0,2549.692383,522.290833,876.766357,0.847926,2.368262,1,CTRL,48.719647,498.701813,202.661972
2,2,WT,CTRL,EC0,1.3.5,"(3, 5)",1,26043.0,2653.458740,524.336243,874.563110,0.860501,3.006327,1,CTRL,52.206007,502.468353,210.382874
3,3,WT,CTRL,EC0,1.3.5,"(3, 5)",1,25605.0,2736.911621,516.952454,876.656799,0.728416,7.674886,1,CTRL,50.552221,501.905182,212.070969
4,4,WT,CTRL,EC0,1.3.5,"(3, 5)",1,26430.0,2490.947266,521.947449,880.909363,0.530618,6.560051,1,CTRL,54.463202,502.564423,204.565842
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22813,70,WT,CTRL,EC0,1015.3.5,"(3, 5)",1015,39979.0,2201.615723,579.509094,651.614807,0.970827,2.473534,1,CTRL,0.000000,378.059143,0.000000
22814,71,WT,CTRL,EC0,1015.3.5,"(3, 5)",1015,38456.0,2243.430664,579.219421,651.633118,0.972632,0.290251,1,CTRL,0.000000,377.311035,0.000000
22815,72,WT,CTRL,EC0,1015.3.5,"(3, 5)",1015,39006.0,2209.442627,578.878113,652.432434,0.968598,0.869137,1,CTRL,0.000000,378.968353,0.000000
22816,73,WT,CTRL,EC0,1015.3.5,"(3, 5)",1015,35816.0,2537.284668,579.752319,652.408752,0.969917,0.874527,1,CTRL,0.000000,379.737701,0.000000


In [286]:
df.to_pickle(f'/mnt/DATA/macrohet/{expt_ID}/results/preliminary_sc_measures/sc_dfs/{acq_ID}_<70_sc_df_GT_dev.pkl')

## Bulk integration of Mtb area measurements

In [ ]:
for acq_ID in tqdm(assay_layout.iterrows(), total = len(assay_layout):
    
    row, column = acq_ID
    tracks_fn = f'/mnt/DATA/macrohet/PS0000/labels/macrohet_seg_model/{acq_ID}.h5'
    dict_fn = f'/mnt/DATA/macrohet/{expt_ID}/upstream_development/tracking/tracking_performance/v1>70/{row},{column}/{acq_ID}_track_assessment.json'
    
    with open(dict_fn) as json_data:
        track_dict = json.load(json_data)
    true_IDs = [int(ID) for ID in track_dict.keys() if track_dict[ID] == True]
    with btrack.io.HDF5FileHandler(tracks_fn, 'r', obj_type='obj_type_1') as reader:
        tracks = reader.tracks
        masks = reader.segmentation
    true_tracks = [t for t in tracks if t.ID in true_IDs]
    
    image_dir = os.path.join(base_dir, 'acquisition/Images')
    images = tile.compile_mosaic(image_dir, 
                                 metadata, 
                                 row, column, 
                                 set_plane='max_proj',
                                 set_channel=2,
                                )
    rfp_images = images[:,0,...].compute().compute()
    
    df = pd.read_pickle(f'/mnt/DATA/macrohet/{expt_ID}/results/preliminary_sc_measures/sc_dfs/{acq_ID}_sc_df_GT_dev.pkl')
    df = df[df['Acquisition ID'] == acq_ID]    
    already_done = set(list(df['Cell ID'].unique()))
    true_IDs = set(true_IDs)
    to_do_IDs = true_IDs.difference(already_done)
    if len(to_do_IDs) == 0:
        print(f'GT track lengths are the same for {acq_ID}')
        continue
    track_IDs = to_do_IDs
    mtb_areas_dict = {}
    dfs = []
    for track_ID in tqdm(track_IDs, total = len(track_IDs)):
        track = [t for t in tracks if t.ID == track_ID][0]
        info = assay_layout.loc[acq_ID]
        mtb_areas = tools.measure_mtb_area(track, masks, rfp_images)
        # initial infection status 
        # init_infec_status = [True for t in track.t if any(mtb_areas[0:5])]
        ### compile single track dictionary of info
        d = {'Time (hours)':track['t'], 
             'Mtb Area µm2': mtb_areas,
             'x':track['x'],
             'y':track['y'],
             # 'Infection status':track['Infected'],
             # 'Initial infection status':track['Infected'][0], 
             # 'Final infection status':track['Infected'][-1], 
             'Mphi Area':track['area'], 
             'Mtb RFP':track['mean_intensity'][:, 1],
             'Mtb RFP threshold':track['mean_intensity'][:, 2],
             'Mphi GFP':track['mean_intensity'][:, 0],
             'Eccentricity':np.sqrt(1-((track['minor_axis_length']**2)/(track['major_axis_length']**2))),
             'MSD': [tools.euc_dist(track['x'][i-1], 
                                    track['y'][i-1], 
                                    track['x'][i], 
                                    track['y'][i]) 
                      if i != 0 else 0
                      for i in range(0, len(track))],
             'Strain':[info['Strain'] for i in range(len(track['t']))], 
             'Compound':[info['Compound'] for i in range(len(track['t']))], 
             'Concentration':[info['ConcentrationEC'] for i in range(len(track['t']))], 
             'Cell ID':[track.ID for i in range(len(track['t']))],
             'Acquisition ID':[acq_ID for i in range(len(track['t']))], 
             'Unique ID': [f'{track.ID}.{acq_ID[0]}.{acq_ID[1]}.' for i in range(len(track['t']))]}
    
        mtb_areas_dict[f'{track.ID}.{acq_ID}'] = mtb_areas
        ### append df to list of dfs
        dfs.append(pd.DataFrame(d))
    ### concat single track dfs into big df
    df2 = pd.concat(dfs, ignore_index=True)
    #rename rd1
    df2['Strain/Compound'] = df2['Compound']  # Create a new column and copy values from 'Compound'
    # Replace values in 'NewColumn' where 'Strain' is 'RD1' with 'RD1'
    df2.loc[df2['Strain'] == 'RD1', 'Strain/Compound'] = 'RD1'
    # save out 
    df2.to_pickle(f'/mnt/DATA/macrohet/{expt_ID}/results/preliminary_sc_measures/sc_dfs/{acq_ID}_sc_df_GT_dev_otherGT_trax.pkl')